In [2]:
# TrustyAI LIME Explanation - Clean Working Code
# ==============================================

# 1. Import Required Libraries
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Import TrustyAI components
import trustyai
from trustyai.explainers import LimeExplainer
from trustyai.model import FeatureFactory, PredictionInput
from trustyai.utils import TestModels
from java.util import Arrays

print(f"✅ TrustyAI version: {trustyai.__version__}")

# 2. Create and Train a Model
# Create synthetic dataset
X, y = make_classification(
    n_samples=1000, 
    n_features=10, 
    n_informative=5,
    n_redundant=2,
    n_classes=2,
    random_state=42
)

# Create feature names
feature_names = [f'feature_{i}' for i in range(X.shape[1])]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

print(f"✅ Model trained - Accuracy: {rf_model.score(X_test, y_test):.3f}")

# 3. Prepare Sample Data for Explanation
sample_size = 5
X_sample = X_test[:sample_size]
y_sample = y_test[:sample_size]

# Convert to TrustyAI PredictionInput format
prediction_inputs = []
for i in range(sample_size):
    features = []
    for j, feature_name in enumerate(feature_names):
        feature = FeatureFactory.newNumericalFeature(feature_name, X_sample[i, j])
        features.append(feature)
    
    pred_input = PredictionInput(features)
    prediction_inputs.append(pred_input)

print(f"✅ Created {len(prediction_inputs)} prediction inputs")

# 4. Set Up TrustyAI Model (using TestModel for demonstration)
# Create a simple linear model for explanation
weights = np.array([1.0, 2.0, -1.0, 0.5, 1.5, -0.5, 0.8, -0.3, 1.2, -0.7])
trusty_model = TestModels.getLinearModel(weights)

print("✅ TrustyAI model created")

# 5. Generate LIME Explanation
# Create LIME explainer
lime_explainer = LimeExplainer()

# Get sample input
sample_input = prediction_inputs[0]

# Generate prediction output from the model
prediction_inputs_list = Arrays.asList([sample_input])
prediction_result = trusty_model.predictAsync(prediction_inputs_list).get()
first_prediction = prediction_result.get(0)

print("✅ Model prediction generated")

# Generate LIME explanation using the correct signature:
# explain(inputs, outputs, model)
lime_result = lime_explainer.explain(sample_input, first_prediction, trusty_model)

print("🎉 LIME explanation generated successfully!")

# 6. Extract and Display Results
print("\n📊 LIME Results:")
print("="*50)

# Method 1: DataFrame format (most useful)
df_result = lime_result.as_dataframe()
if isinstance(df_result, dict):
    for key, value in df_result.items():
        # Convert Java string to Python string safely
        key_name = str(key).upper()
        print(f"\n📈 {key_name} EXPLANATION:")
        if hasattr(value, 'head'):
            print(value.head())
        else:
            print(value)

# Method 2: Raw saliency map (if needed)
try:
    saliency_map = lime_result.saliency_map()
    print(f"\n🔍 Saliency Map Type: {type(saliency_map)}")
    print(f"Available keys: {list(saliency_map.keys()) if saliency_map else 'None'}")
except Exception as e:
    print(f"Saliency map access: {e}")

# 7. Interpretation Summary
print(f"\n🎯 INTERPRETATION:")
print("✅ Successfully generated LIME explanation using TrustyAI")
print("✅ The DataFrame shows:")
print("   - Feature: Input feature names")
print("   - Value: Actual feature values for this prediction")
print("   - Saliency: Importance score (higher = more influential)")
print("   - Confidence: LIME's confidence in the explanation")

print(f"\n🚀 NEXT STEPS:")
print("1. Try with your own sklearn models")
print("2. Experiment with different samples")
print("3. Explore other TrustyAI explainers (SHAP, PDP)")
print("4. Set up model monitoring and bias detection")

# 8. Helper Function for Future Use
def explain_prediction_with_lime(model, sample_input, feature_names):
    """
    Helper function to generate LIME explanations for any model prediction
    
    Args:
        model: TrustyAI PredictionProvider model
        sample_input: TrustyAI PredictionInput object
        feature_names: List of feature names
    
    Returns:
        lime_result: LimeResults object with explanations
    """
    lime_explainer = LimeExplainer()
    
    # Generate prediction
    prediction_list = Arrays.asList([sample_input])
    prediction_output = model.predictAsync(prediction_list).get().get(0)
    
    # Generate explanation
    lime_result = lime_explainer.explain(sample_input, prediction_output, model)
    
    return lime_result

print("\n✅ Helper function 'explain_prediction_with_lime' defined for reuse")

✅ TrustyAI version: 0.6.1
✅ Model trained - Accuracy: 0.945
✅ Created 5 prediction inputs
✅ TrustyAI model created
✅ Model prediction generated
🎉 LIME explanation generated successfully!

📊 LIME Results:

📈 LINEAR-SUM EXPLANATION:
     Feature     Value  Saliency  Confidence
0  feature_8  1.667013  2.921838         0.0
1  feature_0  0.537817  2.806179         0.0
2  feature_4  1.387787  2.639452         0.0
3  feature_6  2.890934  2.522384         0.0
4  feature_1  3.044738  2.655626         0.0

🔍 Saliency Map Type: <class 'dict'>
Available keys: ['linear-sum']

🎯 INTERPRETATION:
✅ Successfully generated LIME explanation using TrustyAI
✅ The DataFrame shows:
   - Feature: Input feature names
   - Value: Actual feature values for this prediction
   - Saliency: Importance score (higher = more influential)
   - Confidence: LIME's confidence in the explanation

🚀 NEXT STEPS:
1. Try with your own sklearn models
2. Experiment with different samples
3. Explore other TrustyAI explainers (SHAP